
## This a code to do a simple RAG using Gemini as Embedder

In [ ]:
!pip install PyPDF2
!pip install -U -q google.generativeai
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key
from google.colab import userdata

import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
genai.configure(api_key="")

In [ ]:
import PyPDF2

# Replace 'path_to_your_pdf.pdf' with the actual path to your PDF file.
path_to_pdf = 'i robot.pdf'

# Function to extract text from PDF using PyPDF2
def extract_text_from_pdf(pdf_path):
    # Open the PDF file in binary read mode
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader (file)
        full_text = ""

        # Iterate through each page of the PDF
        for page_number in range(len(pdf_reader.pages)):
            # Get a page by its number and extract text
            page = pdf_reader.pages[page_number]
            full_text += page.extract_text() + "\n\n"  # Add page breaks

    return full_text

# Function to split text into chunks
def split_text_into_chunks(text, chunk_size=300):  # Adjust the chunk size as needed
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Extract text from the PDF
pdf_text = extract_text_from_pdf(path_to_pdf)

# Split the text into manageable chunks
text_chunks = split_text_into_chunks(pdf_text)

# Example processing of chunks
documents = []
for i, chunk in enumerate(text_chunks):
    # Process each chunk as needed
    documents.append(chunk)
    print(f"Chunk {i+1}:\n{chunk[:50]}...\n")  # Print the first chars of each chunk


Chunk 1:
I, Robot  
Isaac Asimov  
 
 
TO JOHN W. CAMPBELL,...

Chunk 2:
shed in Astounding Science Fiction:  
Reason , cop...

Chunk 3:
ons, Inc.; copyright ©1970 by Isaac 
Asimov.  
Cat...

Chunk 4:
 Street and Smith Publications, Inc.  
The Evitabl...

Chunk 5:
ie ................................ .................

Chunk 6:
....... 20 
Reason ..................................

Chunk 7:
................. 48 
Liar! .........................

Chunk 8:
 ............................... 77 
Escape! ........

Chunk 9:
... ................................ .......... 11...

Chunk 10:
bots and might as 
well have spent them at home wi...

Chunk 11:
the year of Dr. Calvin’s birth that Lawrence Rober...

Chunk 12:
hich 
Dr. Alfred Lanning of U. S. Robots had demon...

Chunk 13:
 took no part in the hectic discussi on period tha...

Chunk 14:
he obtained her bachelor’s degree at Columbia in 2...

Chunk 15:
y globe of plantinumiridium about the size of a hu...

Chunk 16:
ned her Ph.D. and joined

In [ ]:
#indexing in pandas, not the best approach but...
import pandas as pd
df = pd.DataFrame(documents, columns=['text'])
df.head()

,text
0,"I, Robot \nIsaac Asimov \n \n \nTO JOHN W. C..."
1,shed in Astounding Science Fiction: \nReason ...
2,"ons, Inc.; copyright ©1970 by Isaac \nAsimov. ..."
3,"Street and Smith Publications, Inc. \nThe Ev..."
4,ie ................................ .............


In [ ]:
model = 'models/embedding-001'


# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(text):
  # title field is optional.
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                              )["embedding"]

df['Embeddings'] = df['text'].apply(embed_fn)
df

,text,Embeddings
0,"I, Robot \nIsaac Asimov \n \n \nTO JOHN W. C...","[0.049255203, -0.030508304, -0.08248186, -0.00..."
1,shed in Astounding Science Fiction: \nReason ...,"[0.009809838, 0.008349399, -0.10223425, -0.045..."
2,"ons, Inc.; copyright ©1970 by Isaac \nAsimov. ...","[0.015381896, -0.015999457, -0.079194285, -0.0..."
3,"Street and Smith Publications, Inc. \nThe Ev...","[0.02788373, -0.038321562, -0.031660438, -0.04..."
4,ie ................................ .............,"[0.024075534, 0.003053291, -0.08376013, -0.010..."
...,...,...
1368,"swer. We don’t know. Only the Machines know, a...","[0.057336114, -0.0059922454, -0.060653415, -0...."
1369,sociological forces it did not \nunderstand —...,"[0.05148114, 0.005523816, -0.04175465, -0.0007..."
1370,"osal, the absolute control of \nour economy.” ...","[0.034776814, -0.0012511545, -0.044041827, -0...."
1371,"\n— \n \n“And that is all,” said Dr. Calvin,...","[0.026354035, 0.011777443, 0.0007824432, -0.03..."


In [ ]:
query = "Where is explicit that a robot will attack an human?"
query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
query_embedding


{'embedding': [0.029330766,
  -0.07163519,
  -0.024778971,
  -0.007368411,
  0.006249341,
  0.024393518,
  0.039079335,
  -0.013945694,
  -0.028817134,
  0.0434315,
  0.0444018,
  0.0009201791,
  -0.016106755,
  -0.014933657,
  -0.00031433636,
  -0.03674037,
  0.009455876,
  -0.0060656667,
  0.0016117422,
  -0.055706006,
  0.05709117,
  0.022125311,
  -0.036305163,
  -0.009489024,
  0.0174061,
  -0.031979278,
  -0.0034450556,
  -0.069887236,
  0.019516654,
  0.03824869,
  -0.08437349,
  0.037035853,
  -0.067681834,
  -0.036076777,
  -0.011859794,
  -0.05743695,
  0.00683407,
  0.069238305,
  -0.013995151,
  0.02003969,
  -0.00667045,
  0.02283716,
  -0.008595221,
  -0.0006323066,
  0.033336524,
  0.0073230173,
  0.01906229,
  0.066921055,
  0.02302631,
  -0.015183251,
  0.055484597,
  0.04090144,
  0.066536956,
  -0.0035384095,
  -0.014525375,
  -0.06851434,
  0.017419832,
  0.017444083,
  -0.072464466,
  0.02966928,
  0.004609619,
  0.015757024,
  -0.04086784,
  0.049264148,
  -0.0370

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

def find_best_chunks(query_embedding_vector, dataframe,top_n=10):




    # Normalize the embeddings in the dataframe
    normalized_embeddings = normalize(np.stack(dataframe['Embeddings']))

    # Calculate cosine similarities (dot product of normalized vectors)
    cosine_similarities = np.dot(normalized_embeddings, query_embedding_vector)

    # Find the index of the document with the highest cosine similarity
    top_indices = np.argsort(cosine_similarities)[-top_n:][::-1]

    top_passages = dataframe.iloc[top_indices]['text'].tolist()
    top_similarities = cosine_similarities[top_indices]

    return top_passages, top_similarities


In [ ]:

query_embedding_vector = np.array(query_embedding['embedding'])
top_passages,top_maxsimilarity = find_best_chunks(query_embedding_vector, df)


for i in range(len(top_passages)):
  print("Result: ")
  print(top_passages[i])
  print("\nSimilarity: ",top_maxsimilarity[i])



Result: 
ean there is one 
time when a robot may strike a human being without breaking the First Law. Just one time.”  
“And when is that?”  
Dr. Calvin was at the door. She said quietly, “When the human to be struck is merely another 
robot.”  
She smiled broadly, her thin face glowing. “Good -by Mr. Byerle

Similarity:  0.694634535241439
Result: 
ferent matter. It’s a question of the Robotic Laws. The Brain, for insta nce, could never supply a 
solution to a problem set to it if that solution, would involve the death or injury of humans. As far 
as it would be concerned, a problem with only such a solution would be insoluble. If such a 
prob

Similarity:  0.6913298506601677
Result: 
 — I’ll quote it — ‘No robot may harm a human being, or through inaction, allow a 
human being to come to harm.’  
“That’s primary, Dr . Calvin. When it was necessary for one of our men to expose himself for a 
short period to a moderate gamma field, one that would have no physiological effects, the

Simi

In [ ]:
def run_prompt(prompt_text, context_text):
  model = genai.GenerativeModel('gemini-pro')
  response = model.generate_content(f"{prompt_text}: {context_text}", stream=False)

  return response



In [ ]:
prompt = "In this next text i will send you, is the robot dangerous to the human?"
for context in top_passages:
  response = run_prompt(prompt,context)
  print("\n = = = = = = = = = = == = = = = = =\n")
  response.resolve()
  #text = to_markdown(response.text)
  text = response.text
  print(text)
  print("\n = = = = = = = = = = == = = = = = =\n")


 = = = = = = = = = = == = = = = = =

No, the robot is not dangerous to the human.

The text says that there is only one time when a robot may strike a human being without breaking the First Law, and that is when the human to be struck is merely another robot. This means that the robot is not dangerous to humans, as it will not strike them unless they are robots themselves.

 = = = = = = = = = = == = = = = = =


 = = = = = = = = = = == = = = = = =

The provided text does not mention anything about the robot being dangerous to humans. So I cannot answer this question from the provided context.

 = = = = = = = = = = == = = = = = =


 = = = = = = = = = = == = = = = = =

No, the robot is not dangerous to the human.

The quote you provided is from Isaac Asimov's Three Laws of Robotics, which are a set of rules that govern the behavior of robots. The first law states that "a robot may not injure a human being or, through inaction, allow a human being to come to harm." This means that robots 